In [1]:
import os 

In [2]:
%pwd

'/Users/melihaltin/Documents/Development/data-science/Lung-Cancer/Lung-Cancer/research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str

In [5]:
from Lung_Cancer.constants import *
from Lung_Cancer.utils.common import read_yaml , create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            target_column = "LUNG_CANCER"
           
        )

        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        accuracy = accuracy_score(actual, pred)
        recall = recall_score(actual, pred)
        precision = precision_score(actual, pred)
        f1 = f1_score(actual, pred)
        return accuracy, recall, precision, f1

    
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        prediction = model.predict(test_x)

        accuracy, recall, precision, f1 = self.eval_metrics(test_y, prediction)

        scores = {
            "accuracy": accuracy,
            "recall": recall,
            "precision": precision,
            "f1": f1
        }
       
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    print(e)
    print("Model Evaluation failed")
    raise e

[2024-04-22 16:20:17,936]- Lung_Cancer.logging - INFO - yaml file: config/config.yaml loaded successfully
[2024-04-22 16:20:17,938]- Lung_Cancer.logging - INFO - yaml file: params.yaml loaded successfully
[2024-04-22 16:20:17,940]- Lung_Cancer.logging - INFO - yaml file: schema.yaml loaded successfully
[2024-04-22 16:20:17,941]- Lung_Cancer.logging - INFO - created directory at: artifacts
[2024-04-22 16:20:17,942]- Lung_Cancer.logging - INFO - created directory at: artifacts/model_evaluation
[2024-04-22 16:20:18,091]- Lung_Cancer.logging - INFO - json file saved at: artifacts/model_evaluation/metrics.json
